# 网络结构的可视化

## 准备网络和数据

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
from torch.optim import SGD
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
# 使用手写字体数据，准备训练数据集
train_data = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True, # 之使用训练数据集
    # 将数据转化为torch使用的张量，取值范围为[0,1]
    transform = torchvision.transforms.ToTensor(),
    download = False
)

In [3]:
# 定义一个数据加载器
train_loader = Data.DataLoader(
    dataset=train_data,  # 数据集
    batch_size=128,  # 批处理样本大小
    shuffle=True,
    num_workers=2,
)

# 获得一个batch的数据
for step, (b_x, b_y) in enumerate(train_loader):
    if step > 0:
        break

# 输出训练图像的尺寸和标签的尺寸
print(b_x.shape)
print(b_y.shape)

# 准备需要使用的测试数据集
test_data = torchvision.datasets.MNIST(
    root='./data/MNIST',
    train=False,  # 不使用训练数据集
    download=False,
)

# 为数据添加一个通道纬度，并且取值范围缩放到0到1之间
test_data_x = test_data.data.type(torch.FloatTensor) / 255.0
test_data_x = torch.unsqueeze(test_data_x, dim=1)
test_data_y = test_data.targets  # 测试集的标签

print("test_data_x.shape:", test_data_x.shape)
print("test_data_y.shape:", test_data_y.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128])
test_data_x.shape: torch.Size([10000, 1, 28, 28])
test_data_y.shape: torch.Size([10000])


In [4]:
# 搭建一个卷积神经网络
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # 定义第一个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,    # 输入的feature map
                out_channels=16,  # 输出的feature map
                kernel_size=3,    # 卷积核尺寸
                stride=1,         # 卷积核步长
                padding=1,        # 进行填充
            ),
            nn.ReLU(),            # 激活函数
            nn.AvgPool2d(
                kernel_size=2,    # 平均值池化层，使用2x2
                stride=2,         # 池化步长为2
            ),
        )

        # 定义第二个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,    # 输入的feature map
                out_channels=32,  # 输出的feature map
                kernel_size=3,    # 卷积核尺寸
                stride=1,         # 卷积核步长
                padding=1,        # 进行填充
            ),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)     # 最大值池化
        )

        # 定义全连接层
        self.fc = nn.Sequential(
            nn.Linear(
                in_features=32*7*7,  # 输入特征
                out_features=128,   # 输出特征数
            ),
            nn.ReLU(),
            nn.Linear(
                in_features=128,  # 输入特征
                out_features=64,   # 输出特征数
            ),
            nn.ReLU(),
        )

        # 定义分类层
        self.out = nn.Linear(64, 10)

    # 定义网络的向前传播路径
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)   # 展平多维的卷积图层
        x = self.fc(x)
        output = self.out(x)
        return output

In [5]:
# 输出网络结构
MyConvnet = ConvNet()
print(MyConvnet)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1568, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (out): Linear(in_features=64, out_features=10, bias=True)
)


## HiddenLayer库可视化网络

In [6]:
import hiddenlayer as hl

# 可视化卷积神经网络
hl_graph = hl.build_graph(MyConvnet, torch.zeros([1, 1, 28, 28]))
hl_graph.theme = hl.graph.THEMES['blue'].copy()

# 将可视化的网络保存为图片
hl_graph.save('./data/net/chapter4/MyConvnet_hl.png', format='png')

## PyTorchViz库可视化网络

In [9]:
from torchviz import make_dot

# 使用make_dot可视化网络
x = torch.randn(1, 1, 28, 28).requires_grad_(True)
y = MyConvnet(x)
MyConvnetvis = make_dot(y, params=dict(list(MyConvnet.named_parameters())+[('x',x)]))

# 指定文件保存位置
MyConvnetvis.directory = './data/net/chapter4/MyConvnet_vis'
MyConvnetvis.view() # 会自动在当前文件夹生成文件

'data\\net\\chapter4\\MyConvnet_vis\\Digraph.gv.pdf'

# 训练过程的可视化

## tensorboardX中的常用方法

<center><b>tensorboardX常用功能和调用方式</b></center>

| 函数 | 功能 | 用法 |
| :---- | :---- | :---- |
| SummaryWriter() | 创建编写器，保存日志 | writer=SummaryWriter() |
| writer.add_scalar() | 添加标量 | writer.add_scalar('myscalar',value,iteration) |
| writer.add_image()() | 添加图像 | writer.add_image('imresult',x,iteration) |
| writer.add_histogram()() | 添加直方图 | writer.add_histogram('hist',array,iteration) |
| writer.add_graph()() | 添加网络结构 | writer.add_graph(model,input_to_model=None) |
| writer.add_audio()() | 添加音频 | writer.add_audio(tag,audio,iteration,sample_rate) |
| writer.add_text()() | 添加文本 | writer.add_text(tag,text_string,global_step=None) |

## 利用tensorboardX进行可视化

In [14]:
# 从tensorboardX库中导入需要的API
from tensorboardX import SummaryWriter
SumWriter = SummaryWriter(log_dir='./data/log/chapter4')

# 定义优化器
optimizer = torch.optim.Adam(MyConvnet.parameters(), lr=0.0003)
loss_func = nn.CrossEntropyLoss() # 损失函数
train_loss = 0
print_step = 100 # 每经过100次迭代后，输出损失

# 对模型进行迭代训练，对所有的数据训练epoch轮
for epoch in range(5):
    # 对训练数据的加载器进行迭代计算
    for step, (b_x, b_y) in enumerate(train_loader):
        # 计算每个batch的损失
        output = MyConvnet(b_x)          # CNN在训练batch上的输出
        loss = loss_func(output, b_y)    # 交叉熵损失函数
        optimizer.zero_grad()            # 每个迭代的梯度初始化为0
        loss.backward()                  # 损失的后向传播，计算梯度
        optimizer.step()                 # 使用梯度进行优化
        train_loss = train_loss + loss   # 计算损失的累加损失
        
        # 计算迭代次数
        niter = epoch * len(train_loader) + step + 1
        
        # 计算每经过print_step次迭代后的输出
        if niter % print_step == 0:
            # 为日志添加训练集损失函数
            SumWriter.add_scalar('train loss', train_loss.item()/niter, global_step=niter)
            
            # 计算在测试集上的精度
            output = MyConvnet(test_data_x)
            _, pre_lab = torch.max(output, 1)
            acc = accuracy_score(test_data_y, pre_lab)
            
            # 为日志中添加训练数据的可视化图像，使用当前batch的图像
            # 将一个batch的数据进行预处理
            b_x_im = vutils.make_grid(b_x, nrow=12)
            SumWriter.add_image('train image sample', b_x_im, niter)
            
            # 使用直方图可视化网络中参数的分布情况
            for name, param in MyConvnet.named_parameters():
                SumWriter.add_histogram(name, param.data.numpy(), niter)

## 查看tensorboardX可视化结果

## HiddenLayer库可视化训练过程

In [15]:
# 见书93-94页

# 第四章后续见书95-101页